## Financial Web Scraping with Pandas

In [ ]:
import pandas as pd
import datetime as dt

In [ ]:
import pandas_datareader as pdr

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Single stock prices

In [ ]:
start = dt.datetime(2000,1,1)
end = dt.datetime.today()

In [ ]:
df_stock = pdr.get_data_yahoo('AAPL', start,end)

In [ ]:
df_stock.head(1)

In [ ]:
df_stock.tail(1)

In [ ]:
plt.plot(df_stock.index,df_stock['Close'])
plt.show()

### US Treasury

TREAST - U.S. Treasury securities held by the Federal Reserve ( Millions of Dollars )

FYGFDPUN - Federal Debt Held by the Public ( Millions of Dollars )

FDHBFIN - Federal Debt Held by Foreign and International Investors ( Billions of Dollars )

GFDEBTN - Federal Debt: Total Public Debt ( Millions of Dollars )

BAA10Y' - Baa Corporate Bond Yield Relative to Yield on 10-Year


In [ ]:
tsy_tickers = ['DGS1MO', 'DGS3MO', 'DGS1', 'DGS2', 'DGS5', 'DGS7', 'DGS10', 'DGS30',
               'TREAST', # -- U.S. Treasury securities held by the Federal Reserve ( Millions of Dollars )
               'FYGFDPUN', # -- Federal Debt Held by the Public ( Millions of Dollars )
               'FDHBFIN', # -- Federal Debt Held by Foreign and International Investors ( Billions of Dollars )
               'GFDEBTN', # -- Federal Debt: Total Public Debt ( Millions of Dollars )
               'BAA10Y', # -- Baa Corporate Bond Yield Relative to Yield on 10-Year
              ]
tsy_data = pdr.DataReader(tsy_tickers, 'fred').dropna(how='all').ffill()
tsy_data['FDHBFIN'] = tsy_data['FDHBFIN'] * 1000
tsy_data['GOV_PCT'] = tsy_data['TREAST'] / tsy_data['GFDEBTN']
tsy_data['HOM_PCT'] = tsy_data['FYGFDPUN'] / tsy_data['GFDEBTN']
tsy_data['FOR_PCT'] = tsy_data['FDHBFIN'] / tsy_data['GFDEBTN']

In [ ]:
return_period = 5
#Y = tsy_data.loc[:, ['DGS1MO', 'DGS5', 'DGS30']].diff(return_period).shift(-return_period)
#return_period = 5
Y = tsy_data.loc[:, ['DGS1MO', 'DGS5', 'DGS30']].shift(-return_period)
Y.columns = [col+'_pred' for col in Y.columns]

#X = tsy_data.loc[:, ['DGS1MO', 'DGS3MO', 'DGS1', 'DGS2', 'DGS5', 'DGS7', 'DGS10', 'DGS30', 'GOV_PCT', 'HOM_PCT', 'FOR_PCT', 'BAA10Y']].diff(return_period)
X = tsy_data.loc[:, ['DGS1MO', 'DGS3MO', 'DGS1', 'DGS2', 'DGS5', 'DGS7', 'DGS10', 'DGS30', 'GOV_PCT', 'HOM_PCT', 'FOR_PCT', 'BAA10Y']]

dataset = pd.concat([Y, X], axis=1).dropna().iloc[::return_period, :]
Y = dataset.loc[:, Y.columns]
X = dataset.loc[:, X.columns]

In [ ]:
dataset.head(1)

In [ ]:
Y.plot()

In [ ]:

syms = ['DGS10', 'DGS5', 'DGS2', 'DGS1MO', 'DGS3MO']
yc = pdr.DataReader(syms, 'fred') # could specify start date with start param here
names = dict(zip(syms, ['10yr', '5yr', '2yr', '1m', '3m']))
yc = yc.rename(columns=names)
yc = yc[['1m', '3m', '2yr', '5yr', '10yr']]

In [ ]:
yc.loc['2016-06-01'].plot(label='Jun 1')
yc.loc['2016-06-02'].plot(label='Jun 2')
plt.legend(loc=0)

### Web Scaping Treasury website

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

soup = BeautifulSoup(requests.get('https://data.treasury.gov/feed.svc/DailyTreasuryYieldCurveRateData').text,'lxml')
table = soup.find_all('m:properties')
tbondvalues = []
for i in table:
    tbondvalues.append([i.find('d:new_date').text[:10],i.find('d:bc_1month').text,i.find('d:bc_2month').text,i.find('d:bc_3month').text,i.find('d:bc_6month').text,i.find('d:bc_1year').text,i.find('d:bc_2year').text,i.find('d:bc_3year').text,i.find('d:bc_5year').text,i.find('d:bc_10year').text,i.find('d:bc_20year').text,i.find('d:bc_30year').text])
ustcurve = pd.DataFrame(tbondvalues,columns=['date','1m','2m','3m','6m','1y','2y','3y','5y','10y','20y','30y'])
ustcurve.iloc[:,1:] = ustcurve.iloc[:,1:].apply(pd.to_numeric)/100
ustcurve['date'] = pd.to_datetime(ustcurve['date'])

In [ ]:
ustcurve

### Stocks scraping

In [ ]:
from pylab import plot,show
from numpy import vstack,array
from numpy.random import rand
import numpy as np
from scipy.cluster.vq import kmeans,vq
import pandas as pd
import pandas_datareader as dr
from math import sqrt
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt

sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'


#read in the url and scrape ticker data
data_table = pd.read_html(sp500_url)



In [ ]:
ticker_df = data_table[0][1:]

In [ ]:
ticker_df.head()

In [ ]:
tickers = data_table[0][1:]['Symbol'].tolist()
len(tickers)

In [ ]:
import io
import pandas
from datetime import datetime
import requests
from tqdm import tqdm

class YahooData:



    def __init__(self, tickers: list) -> None:
        self._tickers = tickers

    
    def fetch(self, start: str, end: str):

        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'
        }

        prices_df = pd.DataFrame()

        for ticker in self._tickers:
            # print(f"Requesting ticker=[{ticker}]")

            try:
                url = "https://query1.finance.yahoo.com/v7/finance/download/" + str(ticker)
                x = int(datetime.strptime(start, '%Y-%m-%d').strftime("%s"))
                y = int(datetime.strptime(end, '%Y-%m-%d').strftime("%s"))
                url += "?period1=" + str(x) + "&period2=" + str(y) + "&interval=1d&events=history&includeAdjustedClose=true"
            
                r = requests.get(url, headers=headers)
                df = pandas.read_csv(io.StringIO(r.text), index_col=0, parse_dates=['Date'])

                df.reset_index(inplace=True)
            
                df = df.assign(Ticker=ticker)
                prices_df = prices_df.append(df, ignore_index=True)
                
                print(f"Number of records for ticker=[{ticker}] is {prices_df.shape[0]}")
            except:
                pass
                
        return prices_df


start = dt.datetime(2013,1,1)
end = dt.datetime.today()

yahoo_data = YahooData(tickers=tickers)

start_str = start.strftime('%Y-%m-%d')
end_str = end.strftime('%Y-%m-%d')
yahoo_data_df = yahoo_data.fetch(start=start_str, end=end_str)

In [ ]:
yahoo_data_df.to_csv('sp500.csv', index=False)

In [80]:
start = dt.datetime(2018,1,1)
end = dt.datetime.today()

yahoo_data = YahooData(tickers=tickers)

start_str = start.strftime('%Y-%m-%d')
end_str = end.strftime('%Y-%m-%d')
yahoo_data_df = yahoo_data.fetch(start=start_str, end=end_str)

Number of records for ticker=[AOS] is 1380
Number of records for ticker=[ABT] is 2760
Number of records for ticker=[ABBV] is 4140
Number of records for ticker=[ACN] is 5520
Number of records for ticker=[ATVI] is 6900
Number of records for ticker=[ADM] is 8280
Number of records for ticker=[ADBE] is 9660
Number of records for ticker=[ADP] is 11040
Number of records for ticker=[AAP] is 12420
Number of records for ticker=[AES] is 13800
Number of records for ticker=[AFL] is 15180
Number of records for ticker=[A] is 16560
Number of records for ticker=[APD] is 17940
Number of records for ticker=[AKAM] is 19320
Number of records for ticker=[ALK] is 20700
Number of records for ticker=[ALB] is 22080
Number of records for ticker=[ARE] is 23460
Number of records for ticker=[ALGN] is 24840
Number of records for ticker=[ALLE] is 26220
Number of records for ticker=[LNT] is 27600
Number of records for ticker=[ALL] is 28980
Number of records for ticker=[GOOGL] is 30360
Number of records for ticker=[GOO

In [81]:
yahoo_data_df.to_csv('sp500-5yrs.csv', index=False)